In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
import statistics
import math
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [2]:
k20 = pd.read_csv( "kaggle2020.csv" )
k20 = pd.DataFrame(k20)
k21 = pd.read_csv( "kaggle2021.csv" )
k21 = pd.DataFrame(k21)
k22 = pd.read_csv( "kaggle2022.csv" )
k22 = pd.DataFrame(k22)

/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_89905/2393232715.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  k20 = pd.read_csv( "kaggle2020.csv" )
/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_89905/2393232715.py:3: DtypeWarning: Columns (0,195,201,285,286,287,288,289,290,291,292) have mixed types. Specify dtype option on import or set low_memory=False.
  k21 = pd.read_csv( "kaggle2021.csv" )
/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_89905/2393232715.py:5: DtypeWarning: Columns (0,208,225,255,257,260,270,271,277) have mixed types. Specify dtype option on import or set low_memory=False.
  k22 = pd.read_csv( "kaggle2022.csv" )


In [3]:
#filter necessary columns, create new column to indicate year for each dataframe
new20 = k20[['Q1','Q2','Q24', 'Q3', 'Q6','Q4','Q5','Q7_Part_1','Q7_Part_2','Q7_Part_3','Q7_Part_4','Q7_Part_5','Q7_Part_6','Q7_Part_7','Q7_Part_8','Q7_Part_9', 'Q7_Part_10','Q7_Part_11','Q7_Part_12','Q7_OTHER', 'Q14_Part_1','Q14_Part_2','Q14_Part_3','Q14_Part_4','Q14_Part_5','Q14_Part_6','Q14_Part_7','Q14_Part_8','Q14_Part_9','Q14_Part_10','Q14_OTHER','Q15', 'Q22','Q25']]
new20.insert(0, "Survey Year", 2020)
new21 = k21[['Q1','Q2','Q25','Q3', 'Q6','Q4','Q5','Q7_Part_1','Q7_Part_2','Q7_Part_3','Q7_Part_4','Q7_Part_5','Q7_Part_6','Q7_Part_7','Q7_Part_8','Q7_Part_9', 'Q7_Part_10','Q7_Part_11','Q7_Part_12','Q7_OTHER', 'Q14_Part_1','Q14_Part_2','Q14_Part_3','Q14_Part_4','Q14_Part_5','Q14_Part_6','Q14_Part_7','Q14_Part_8','Q14_Part_9','Q14_Part_10','Q14_OTHER','Q15', 'Q23','Q26']]
new21.insert(0, "Survey Year", 2021)
new22 = k22[['Q2','Q3','Q29', 'Q4', 'Q11','Q8','Q23','Q12_1','Q12_2','Q12_3','Q12_4','Q12_5','Q12_6','Q12_7','Q12_8','Q12_9','Q12_10', 'Q12_11','Q12_12','Q12_13','Q12_14','Q12_15', 'Q15_1', 'Q15_2','Q15_3','Q15_4','Q15_5','Q15_6','Q15_7','Q15_8','Q15_9','Q15_10', 'Q15_15','Q16', 'Q27', 'Q30']]
new22.insert(0, "Survey Year", 2022)

#2020 and 2021 does not include C# and GO options, while 2022 does not include Swift option, 
# sub these options into the “Others” category
current = 0
for row in new22.itertuples():
    a = row.Q12_5 
    b = row.Q12_13 
    if a == 'C#' or b =='Go':
        new22.loc[current,'Q12_15'] = 'Other'
    current += 1
current = 0
for row in new21.itertuples():
    a = row.Q7_Part_9  
    if a == 'Swift':
        new21.loc[current,'Q7_OTHER'] = 'Other'
    current += 1
current = 0
for row in new20.itertuples():
    a = row.Q7_Part_9  
    if a == 'Swift':
        new20.loc[current,'Q7_OTHER'] = 'Other'
    current += 1
    
#remove certain columns
new20 = new20[['Survey Year','Q1','Q2','Q24', 'Q3', 'Q6','Q4','Q5','Q7_Part_1','Q7_Part_2','Q7_Part_3','Q7_Part_4','Q7_Part_5','Q7_Part_6','Q7_Part_7','Q7_Part_8', 'Q7_Part_10','Q7_Part_11','Q7_OTHER', 'Q14_Part_1','Q14_Part_2','Q14_Part_3','Q14_Part_4','Q14_Part_5','Q14_Part_6','Q14_Part_7','Q14_Part_8','Q14_Part_9','Q14_Part_10','Q14_OTHER','Q15', 'Q22', 'Q25']]
new21 = new21[['Survey Year','Q1','Q2','Q25','Q3', 'Q6','Q4','Q5','Q7_Part_1','Q7_Part_2','Q7_Part_3','Q7_Part_4','Q7_Part_5','Q7_Part_6','Q7_Part_7','Q7_Part_8', 'Q7_Part_10','Q7_Part_11','Q7_OTHER', 'Q14_Part_1','Q14_Part_2','Q14_Part_3','Q14_Part_4','Q14_Part_5','Q14_Part_6','Q14_Part_7','Q14_Part_8','Q14_Part_9','Q14_Part_10','Q14_OTHER','Q15', 'Q23','Q26']]
new22 = new22[['Survey Year','Q2','Q3','Q29', 'Q4', 'Q11','Q8','Q23','Q12_1','Q12_2','Q12_3','Q12_4','Q12_6','Q12_7','Q12_8','Q12_12','Q12_9', 'Q12_11','Q12_15', 'Q15_1', 'Q15_2','Q15_3','Q15_4','Q15_5','Q15_6','Q15_7','Q15_8','Q15_9','Q15_10','Q15_15','Q16','Q27','Q30']]

#rename 2022 columns so that all columns names are the same
new20 = new20.rename(columns={'Survey Year':'Year','Q1':'Age','Q2':'Gender','Q24':'Income','Q3':'Country','Q6':'Experience','Q4':'Education','Q5':'Job Role','Q7_Part_1':'Python','Q7_Part_2':'R','Q7_Part_3':'SQL','Q7_Part_4':'C','Q7_Part_5':'C++','Q7_Part_6':'Java','Q7_Part_7':'Javascript','Q7_Part_8':'Julia','Q7_Part_10':'Bash','Q7_Part_11':'MATLAB','Q7_OTHER':'Other Lang', 'Q14_Part_1':'Matplotlib', 'Q14_Part_2':'Seaborn','Q14_Part_3':'Ploty','Q14_Part_4':'Ggplot','Q14_Part_5':'Shiny','Q14_Part_6':'D3','Q14_Part_7':'Altair','Q14_Part_8':'Bokeh','Q14_Part_9':'Geoplotlib','Q14_Part_10':'Leaflet','Q14_OTHER':'Other Vis','Q15':'ML Years','Q22':'ML in Job','Q25':'Spending on ML'})
new21 = new21.rename(columns={'Survey Year':'Year','Q1':'Age','Q2':'Gender','Q25':'Income','Q3':'Country','Q6':'Experience','Q4':'Education','Q5':'Job Role','Q7_Part_1':'Python','Q7_Part_2':'R','Q7_Part_3':'SQL','Q7_Part_4':'C','Q7_Part_5':'C++','Q7_Part_6':'Java','Q7_Part_7':'Javascript','Q7_Part_8':'Julia','Q7_Part_10':'Bash','Q7_Part_11':'MATLAB','Q7_OTHER':'Other Lang', 'Q14_Part_1':'Matplotlib', 'Q14_Part_2':'Seaborn','Q14_Part_3':'Ploty','Q14_Part_4':'Ggplot','Q14_Part_5':'Shiny','Q14_Part_6':'D3','Q14_Part_7':'Altair','Q14_Part_8':'Bokeh','Q14_Part_9':'Geoplotlib','Q14_Part_10':'Leaflet','Q14_OTHER':'Other Vis','Q15':'ML Years','Q23':'ML in Job','Q26':'Spending on ML'})
new22 = new22.rename(columns={'Survey Year':'Year','Q2':'Age','Q3':'Gender','Q29':'Income','Q4':'Country','Q11':'Experience','Q8':'Education','Q23':'Job Role','Q12_1':'Python','Q12_2':'R','Q12_3':'SQL','Q12_4':'C','Q12_6':'C++','Q12_7':'Java','Q12_8':'Javascript','Q12_12':'Julia','Q12_9':'Bash', 'Q12_11':'MATLAB','Q12_15':'Other Lang','Q15_1':'Matplotlib', 'Q15_2':'Seaborn','Q15_3':'Ploty','Q15_4':'Ggplot','Q15_5':'Shiny','Q15_6':'D3','Q15_7':'Altair','Q15_8':'Bokeh','Q15_9':'Geoplotlib','Q15_10':'Leaflet','Q15_15':'Other Vis','Q16':'ML Years', 'Q27':'ML in Job','Q30':'Spending on ML'})


#concat the 3 dataframes together

data2 = pd.concat([new20,new21])
data = pd.concat([data2,new22])

data = data.tail(-1)
data.reset_index(drop=True, inplace=True)

print(data.shape)

#group certain age categories together 
data['Age'] = data['Age'].str.replace("18-21","18-24")
data['Age'] = data['Age'].str.replace("22-24","18-24")
#group certain age categories together 
data['Experience'] = data['Experience'].str.replace("1-2 years","1-3 years")
#group certain role categories together 
data['Job Role'] = data['Job Role'].str.replace("Product/Project Manager","Manager")
data['Job Role'] = data['Job Role'].str.replace("Program/Project Manager","Manager")
data['Job Role'] = data['Job Role'].str.replace("Product Manager","Manager")
data['Job Role'] = data['Job Role'].str.replace("Manager (Program, Project, Operations, Executive-level, etc)","Manager", regex=False)
data['Job Role'] = data['Job Role'].str.replace("Machine Learning Engineer","ML Engineer")
data['Job Role'] = data['Job Role'].str.replace("Machine Learning/ MLops Engineer","ML Engineer")
data['Job Role'] = data['Job Role'].str.replace("Data Analyst (Business, Marketing, Financial, Quantitative, etc)","Data Analyst", regex=False)
data['Job Role'] = data['Job Role'].str.replace("Developer Advocate","Developer Relations/Advocacy")
data['Income'] = data['Income'].str.replace("$500,000-999,999","> $500,000", regex=False)
data['Income'] = data['Income'].str.replace(">$1,000,000","> $500,000", regex=False)
data['Income'] = data['Income'].str.replace("300,000-499,999","300,000-500,000", regex=False)

#replacing other minority gender categories as "Others"
data['Gender'] = data['Gender'].replace(['Prefer not to say', 'Nonbinary', 'Prefer to self-describe'], 'Others')


data = data.drop(data[data['Job Role'] == 'Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].index)                 

(70008, 33)


In [4]:
#dropping data points with irrelevant job roles
data = data.dropna(subset=['Job Role'])
data = data.drop(data[data['Job Role']=='Student'].index)
data = data.drop(data[data['Job Role']=='Currently not employed'].index)
data = data.drop(data[data['Job Role']=='Other'].index)

unique_job_roles = data['Job Role'].unique()

In [5]:
pl_list = ['Python', 'R', 'SQL', 'C' , 'C++', 'Java', 'Javascript', 'Julia', 'Bash', 'MATLAB', 'Other Lang']

country_list_all_data = data['Country']
unique_country_list = country_list_all_data.unique()

dv_tools_list = ['Matplotlib', 'Seaborn','Ploty','Ggplot','Shiny','D3','Altair','Bokeh','Geoplotlib','Leaflet','Other Vis']

gender_list = data['Gender'].unique()

In [6]:
#Encoding the categorical variables into numerical variables
class IncomeTransformer:
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        
        proportions = df['Income'].value_counts(normalize=True)
        unique_income_ranges = proportions.index
        df['Income'] = df['Income'].fillna(pd.Series(np.random.choice(unique_income_ranges,p=proportions,size=len(df))))
        
        OHE_Income = []
        
        for index, row in df.iterrows():
            row_income_str = row['Income']
            if(row_income_str == "$0-999"):
                OHE_Income.append(500)
            elif(row_income_str == "> $500,000"):
                OHE_Income.append(500000)
            else:
                row_income_str = row_income_str.replace(",","")
                row_income_str = row_income_str.strip()
                lower, upper = row_income_str.split("-")
                mean_income = (float(lower)+float(upper))/2
                OHE_Income.append(mean_income)
                
        OHE_Income = np.array(OHE_Income)
        
        min_val = np.min(OHE_Income)
        max_val = np.max(OHE_Income)
        print("Min Income ",min_val)
        print("Max Income ",max_val)
        
        OHE_Income = (OHE_Income - min_val) / (max_val - min_val)
        
        OHE_Income = OHE_Income.reshape(-1,1)
                
        return OHE_Income

In [7]:
#Binning the experience to numerical mean in the range
class ExpTransformer:
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        
        #print(df['Experience'].value_counts(dropna=False))
        
        proportions = df['Experience'].value_counts(normalize=True)
        unique_age_ranges = proportions.index
        df['Experience'] = df['Experience'].fillna(pd.Series(np.random.choice(unique_age_ranges,p=proportions,size=len(df))))

        #print(df['Experience'].value_counts(dropna=False))
        
        OHE_Exp = []
        
        for index, row in df.iterrows():
            row_exp_str = row['Experience']
            if(row_exp_str == "I have never written code"):
                OHE_Exp.append(0)
            elif(row_exp_str == "< 1 years"):
                OHE_Exp.append(0.5)
            elif(row_exp_str == "20+ years"):
                OHE_Exp.append(20)
            else:
                row_exp_str = row_exp_str.replace("years","")
                row_exp_str = row_exp_str.strip()
                lower, upper = row_exp_str.split("-")
                mean_exp = (float(lower)+float(upper))/2
                OHE_Exp.append(mean_exp)
                
        OHE_Exp = np.array(OHE_Exp)
        OHE_Exp = OHE_Exp.reshape(-1,1)
        
        print("Shape of the OHE_Exp: ", OHE_Exp.shape)
                
        return OHE_Exp

In [8]:
#Binning the age values to numerical mean in the range
class AgeTransformer:
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        
        OHE_Age = []
        
        for index, row in df.iterrows():
            row_age_str = row['Age']
            if(row_age_str == "70+"):
                OHE_Age.append(75)
            else:
                lower, upper = row_age_str.split("-")
                mean_age = (float(lower)+float(upper))/2
                OHE_Age.append(mean_age)
                
        OHE_Age = np.array(OHE_Age)
        
        min_val = np.min(OHE_Age)
        max_val = np.max(OHE_Age)
        OHE_Age = (OHE_Age - min_val) / (max_val - min_val)
        
        OHE_Age = OHE_Age.reshape(-1,1)
        
        print("Shape of the OHE_Age: ", OHE_Age.shape)
        
        print(min_val)
        print(max_val)
                
        return OHE_Age

In [9]:
#Ordinal encoding for education column in data
class EduTransformer:
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        
        OHE_Edu = []
        
        for index, row in df.iterrows():
            if(df.at[index,'Education'] == "I prefer not to answer"):
                OHE_Edu.append(0)
            elif(df.at[index,'Education'] == "No formal education past high school"):
                OHE_Edu.append(1)
            elif(df.at[index,'Education'] == "Some college/university study without earning a bachelor’s degree"):
                OHE_Edu.append(2)
            elif(df.at[index,'Education'] == "Bachelor’s degree"):
                OHE_Edu.append(3)
            elif(df.at[index,'Education'] == "Master’s degree"):
                OHE_Edu.append(4)
            elif(df.at[index,'Education'] == "Professional degree"):
                OHE_Edu.append(4)
            elif(df.at[index,'Education'] == "Doctoral degree"):
                OHE_Edu.append(5)
            elif(df.at[index,'Education'] == "Professional doctorate"):
                OHE_Edu.append(6)
                
        OHE_Edu = np.array(OHE_Edu)
        OHE_Edu = OHE_Edu.reshape(-1,1)
        
        print("Shape of the OHE_Edu: ", OHE_Edu.shape)
        
        return OHE_Edu
    

In [10]:
#Encoding the Known_PL

class PLTransformer:
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        
        OHE_PL = []
        
        for index, row in df.iterrows():
            row_PL = []
            for item in pl_list:
                if(pd.isna(df.at[index,item])):
                    row_PL.append(0)
                else:
                    row_PL.append(1)
            OHE_PL.append(row_PL)
            
        OHE_PL = np.array(OHE_PL)
        
        print("Shape of the OHE_PL: ", OHE_PL.shape)
                
        return OHE_PL

In [11]:
#Encoding the countries

class CountryTransformer:  
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        nrows = len(df)
        ncolumns = len(unique_country_list)
        
        df_Country = pd.DataFrame(0, index=range(nrows), columns=unique_country_list)
        OHE_Country = []
        
        df = df.reset_index(drop=True)
        
        for index, row in df.iterrows():
            row_Country = row['Country']
            df_Country.at[index,row_Country] = 1
            
        OHE_Country = df_Country.to_numpy()
        
        print("Shape of the OHE_Country: ", OHE_Country.shape)
        
        return OHE_Country

In [12]:
#Encoding the Data Visualisation tools

class DataVisTransformer:  
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        
        OHE_DV = []
        
        for index, row in df.iterrows():
            row_DV_count = 0
            for item in dv_tools_list:
                if not (pd.isna(df.at[index,item])):
                    row_DV_count = row_DV_count + 1
                
            OHE_DV.append(row_DV_count)
            
        OHE_DV = np.array(OHE_DV)
        OHE_DV = OHE_DV.reshape(-1,1)
        
        print("Shape of the OHE_DV: ", OHE_DV.shape)
                
        return OHE_DV


In [13]:
#Encoding the ML Years

class MLYearsTransformer:
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        
        #print(df['ML Years'].value_counts(dropna=False))
        
        proportions = df['ML Years'].value_counts(normalize=True)
        unique_ml_year_ranges = proportions.index
        df['ML Years'] = df['ML Years'].fillna(pd.Series(np.random.choice(unique_ml_year_ranges,p=proportions,size=len(df))))
        
        #print(unique_ml_year_ranges)
        #print(df['ML Years'].value_counts(dropna=False))
        
        OHE_MLY = []
        
        for index, row in df.iterrows():
            row_MLY_str = row['ML Years']
            if(row_MLY_str == "I do not use machine learning methods"):
                OHE_MLY.append(0)
            elif(row_MLY_str == "Under 1 year"):
                OHE_MLY.append(0.5)
            elif(row_MLY_str == "20 or more years"):
                OHE_MLY.append(20)
            else:
                #print(row_MLY_str)
                row_MLY_str = row_MLY_str.replace("years","")
                row_MLY_str = row_MLY_str.strip()
                lower, upper = row_MLY_str.split("-")
                mean_exp = (float(lower)+float(upper))/2
                OHE_MLY.append(mean_exp)
                
        OHE_MLY = np.array(OHE_MLY)
        
        min_val = np.min(OHE_MLY)
        max_val = np.max(OHE_MLY)
        OHE_MLY = (OHE_MLY - min_val) / (max_val - min_val)
        
        print(min_val)
        print(max_val)
        
        OHE_MLY = OHE_MLY.reshape(-1,1)
        
        print("Shape of the OHE_MLY: ", OHE_MLY.shape)
                
        return OHE_MLY

In [14]:
#Encoding Gender

class GenderTransformer:  
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        nrows = len(df)
        
        df_Gender = pd.DataFrame(0, index=range(nrows), columns=gender_list)
        OHE_Gender = []
        
        df = df.reset_index(drop=True)
        
        for index, row in df.iterrows():
            row_Gender = row['Gender']
            df_Gender.at[index,row_Gender] = 1
            
        OHE_Gender = df_Gender.to_numpy()
        
        print("Shape of the OHE_Country: ", OHE_Gender.shape)
        
        return OHE_Gender

In [15]:
#Ordinal encoding for ML incoporation in job
class MLinJobTransformer:
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        
        proportions = df['ML in Job'].value_counts(normalize=True)
        unique_mlinjob_ranges = proportions.index
        df['ML in Job'] = df['ML in Job'].fillna(pd.Series(np.random.choice(unique_mlinjob_ranges,p=proportions,size=len(df))))
        
        OHE_MLinJob = []
        
        for index, row in df.iterrows():
            row_MLinJob = row['ML in Job']
            #print(row_MLinJob)
            row_MLinJob = row_MLinJob.strip()
            if(row_MLinJob == "I do not know" or row_MLinJob == "No (we do not use ML methods)"):
                OHE_MLinJob.append(0)
            elif(row_MLinJob == "We are exploring ML methods (and may one day put a model into production)"):
                OHE_MLinJob.append(1)
            elif(row_MLinJob == "We use ML methods for generating insights (but do not put working models into production)"):
                OHE_MLinJob.append(2)
            elif(row_MLinJob == "We recently started using ML methods (i.e., models in production for less than 2 years)"):
                OHE_MLinJob.append(3)
            elif(row_MLinJob == "We have well established ML methods (i.e., models in production for more than 2 years)"):
                OHE_MLinJob.append(4)
                
        OHE_MLinJob = np.array(OHE_MLinJob)
        OHE_MLinJob = OHE_MLinJob.reshape(-1,1)
        
        print("Shape of the OHE_MLinJob: ", OHE_MLinJob.shape)
        
        return OHE_MLinJob

In [16]:
#Ordinal Encoding for Spending on ML
class SpendingonMLTransformer:
    def fit(self, df, y = None, **fit_params):
        
        return df
    
    def transform(self, df, y = None, **fit_params):
        
        proportions = df['Spending on ML'].value_counts(normalize=True)
        unique_spendingonjob_ranges = proportions.index
        df['Spending on ML'] = df['Spending on ML'].fillna(pd.Series(np.random.choice(unique_spendingonjob_ranges,p=proportions,size=len(df))))

        #df['Spending on ML'] = df['Spending on ML'].fillna('$0 ($USD)')
        
        #print(df['Spending on ML'].value_counts(dropna=False))
        
        OHE_SpendinML = []
        
        for index, row in df.iterrows():
            row_SpendinML = row['Spending on ML']
            #print(row_MLinJob)
            row_SpendinML = row_SpendinML.strip()
            if(row_SpendinML == "$0 ($USD)"):
                OHE_SpendinML.append(0)
            elif(row_SpendinML == "$100,000 or more ($USD)"):
                OHE_SpendinML.append(100000)
            else:
                #print(row_SpendinML)
                row_SpendinML = row_SpendinML.replace("$","")
                row_SpendinML = row_SpendinML.replace(",","")
                row_SpendinML = row_SpendinML.strip()
                lower, upper = row_SpendinML.split("-")
                mean_spend = (float(lower)+float(upper))/2
                #print(mean_spend)
                OHE_SpendinML.append(mean_spend)
        
        OHE_SpendinML = np.array(OHE_SpendinML)
        min_val = np.min(OHE_SpendinML)
        max_val = np.max(OHE_SpendinML)
        OHE_SpendinML = (OHE_SpendinML - min_val) / (max_val - min_val)
        
        print(min_val)
        print(max_val)
        
        OHE_SpendinML = OHE_SpendinML.reshape(-1,1)
        
        print("Shape of the OHE_SpendinML: ", OHE_SpendinML.shape)
        
        return OHE_SpendinML

In [17]:
featureTransformer = FeatureUnion([
    ('Experience', Pipeline([('exp', ExpTransformer())])),
    ('Age', Pipeline([('age', AgeTransformer())])),
    ('Education', Pipeline([('edu', EduTransformer())])),
    ('Programming_Languages', Pipeline([('PL', PLTransformer())])),
    ('Country', Pipeline([('Country', CountryTransformer())])),
    ('Data_Visualisation', Pipeline([('DV', DataVisTransformer())])),
    ('ML_Years', Pipeline([('ML_Years', MLYearsTransformer())])),
    ('Gender', Pipeline([('Gender', GenderTransformer())])),
    ('ML_in_Job', Pipeline([('ML_in_Job', MLinJobTransformer())])),
    ('Spending_on_ML', Pipeline([('Spending_on_ML', SpendingonMLTransformer())]))  
    ])

In [57]:
def getTransformedData(df):
    
    print("Entered getTransformedData Function")
    
    featureTransformer = FeatureUnion([
    ('Experience', Pipeline([('exp', ExpTransformer())])),
    ('Age', Pipeline([('age', AgeTransformer())])),
    ('Education', Pipeline([('edu', EduTransformer())])),
    ('Programming_Languages', Pipeline([('PL', PLTransformer())])),
    ('Country', Pipeline([('Country', CountryTransformer())])),
    ('Data_Visualisation', Pipeline([('DV', DataVisTransformer())])),
    ('ML_Years', Pipeline([('ML_Years', MLYearsTransformer())])),
    ('Gender', Pipeline([('Gender', GenderTransformer())])),
    ('ML_in_Job', Pipeline([('ML_in_Job', MLinJobTransformer())])),
    ('Spending_on_ML', Pipeline([('Spending_on_ML', SpendingonMLTransformer())])),
    ('Income', Pipeline([('Spending_on_ML', IncomeTransformer())]))
    ])
    
    print("FeatureUnion Performed")
    featureTransformer.fit(df)
    print("FeatureTransformer Fit Performed")
    transformed_data = featureTransformer.transform(df)

    return transformed_data

In [58]:
def getUserProfile_FV(user_profile):
    
    user_profile_FV = []
    user_profile_FV.append(user_profile[0])
    
    #Transforming the age in the user profile
    user_profile_age = user_profile[1]
    min_val = 21
    max_val = 75
    user_profile_age = (user_profile_age - min_val) / (max_val - min_val)
    user_profile_FV.append(user_profile_age)
    
    #Transforming the education in the user profile
    user_profile_edu_df = pd.DataFrame(0,index=range(1), columns=['Education'])
    user_profile_edu_df.at[0,'Education'] = user_profile[2].strip()
    user_profile_edt = edt.transform(user_profile_edu_df)
    user_profile_FV.append(user_profile_edt[0,0])
    
    #Transforming the PL in the user profile
    user_profile_PL_list = user_profile[3]
    pl_series = pd.Series(0, index=pl_list)
    for pl in user_profile_PL_list:
        pl_series[pl] = 1
    for item in pl_series:
        user_profile_FV.append(item)
        
    #Transforming the Country in the user profile
    user_profile_cn = user_profile[4]
    country_series = pd.Series(0, index=unique_country_list)
    country_series[user_profile_cn] = 1
    for item in country_series:
        user_profile_FV.append(item)
        
    #Transforming the DV in the user profile
    user_profile_dv = user_profile[5]
    user_profile_FV.append(len(user_profile_dv))
    
    #Transforming the ML Years in the user profile
    user_profile_MLinYears = user_profile[6]
    if(user_profile_MLinYears == "I do not use machine learning methods"):
        user_profile_MLinYears_num = 0
    elif(user_profile_MLinYears == "Under 1 year"):
        user_profile_MLinYears_num = 0.5
    elif(user_profile_MLinYears == "20 or more years"):
        user_profile_MLinYears_num = 20
    else:
        user_profile_MLinYears = user_profile_MLinYears.replace("years","")
        user_profile_MLinYears = user_profile_MLinYears.strip()
        lower, upper = user_profile_MLinYears.split("-")
        mean_exp = (float(lower)+float(upper))/2
        user_profile_MLinYears_num = mean_exp
    min_val = 0
    max_val = 20
    user_profile_MLinYears_num = (user_profile_MLinYears_num - min_val) / (max_val - min_val)
    user_profile_FV.append(user_profile_MLinYears_num)
        
    #Transforming Gender in the user profile
    user_profile_Gender = user_profile[7]
    gender_series = pd.Series(0, index=gender_list)
    print(len(gender_series))
    print(gender_series)
    gender_series[user_profile_Gender] = 1
    print(len(gender_series))
    print(gender_series)
    for item in gender_series:
        user_profile_FV.append(item)
    
    #Transforming ML in Job in the user profile
    user_profile_MLinJob = user_profile[8]
    user_profile_MLinJob = user_profile_MLinJob.strip()
    if(user_profile_MLinJob == "I do not know" or user_profile_MLinJob == "No (we do not use ML methods)"):
        user_profile_MLinJob_num = 0
    elif(user_profile_MLinJob == "We are exploring ML methods (and may one day put a model into production)"):
        user_profile_MLinJob_num = 1
    elif(user_profile_MLinJob == "We use ML methods for generating insights (but do not put working models into production)"):
        user_profile_MLinJob_num = 2
    elif(user_profile_MLinJob == "We recently started using ML methods (i.e., models in production for less than 2 years)"):
        user_profile_MLinJob_num = 3
    elif(user_profile_MLinJob == "We have well established ML methods (i.e., models in production for more than 2 years)"):
        user_profile_MLinJob_num = 4
    user_profile_FV.append(user_profile_MLinJob_num)
    
    #Transforming Spending on ML in the user profile
    user_profile_SpendingonML = user_profile[9]
    user_profile_SpendingonML = user_profile_SpendingonML.strip()
    if(user_profile_SpendingonML == "$0 ($USD)"):
        user_profile_SpendingonML_num = 0
    elif(user_profile_SpendingonML == "$100,000 or more ($USD)"):
        user_profile_SpendingonML_num = 100000
    else:
        user_profile_SpendingonML = user_profile_SpendingonML.replace("$","")
        user_profile_SpendingonML = user_profile_SpendingonML.replace(",","")
        user_profile_SpendingonML = user_profile_SpendingonML.strip()
        lower, upper = user_profile_SpendingonML.split("-")
        mean_spend = (float(lower)+float(upper))/2
        user_profile_SpendingonML_num = mean_spend       
    min_val = 0
    max_val = 100000
    user_profile_SpendingonML_num = (user_profile_SpendingonML_num - min_val) / (max_val - min_val)
    user_profile_FV.append(user_profile_SpendingonML_num)          
    
    
    
    user_profile_FV = np.array(user_profile_FV)
    user_profile_FV = user_profile_FV.reshape(-1,1)
    user_profile_FV = user_profile_FV.T
    
    return user_profile_FV

# Train and Test Split

In [59]:
#Encoding the X_train and X_test so that they can be used to train and test the model.

y_labels = data['Job Role']

X_train, X_test, y_train, y_test = train_test_split(data, y_labels, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(X_train.iloc[3])

X_train_encoded = getTransformedData(X_train)
print(X_train_encoded.shape)

X_test_encoded = getTransformedData(X_test)
print(X_test_encoded.shape)

print(X_train_encoded[3])

X_test_file = X_test.iloc[:20]
X_test_file = X_test_file.drop('Job Role', axis=1)
X_test_file.to_csv("test_data.csv", index=False)

(27160, 33)
(6791, 33)
(27160,)
(6791,)
Year                                                           2021
Age                                                           50-54
Gender                                                          Man
Income                                              200,000-249,999
Country                                    United States of America
Experience                                                20+ years
Education                                           Master’s degree
Job Role                                             Data Scientist
Python                                                       Python
R                                                                 R
SQL                                                             SQL
C                                                               NaN
C++                                                             NaN
Java                                                            NaN
Javascri

In [60]:
def getIncomeforRole(Job_Role):

    job_role_income = X_train[X_train['Job Role'] == Job_Role]['Income']
    print(len(job_role_income))
    job_role_income = [x for x in job_role_income if x is not None]
    print(len(job_role_income))
    income_num = []
    for item in job_role_income:
        if(item == "$0-999"):
            income_num.append(500)
        elif(item == "> $500,000"):
            income_num.append(500000)
        elif(type(item)=='str'):
            item = item.replace(",","")
            item = item.strip()
            lower, upper = item.split("-")
            mean_income = (float(lower)+float(upper))/2
            income_num.append(mean_income)             
                
    median_income = statistics.median(income_num)
    print("The median income for this role: ",median_income)
    print("Average income is: ", sum(income_num)/len(income_num))

In [61]:
getIncomeforRole('Manager')

2144
2144
The median income for this role:  500.0
Average income is:  51678.27868852459


# PCA Transformation

In [62]:
pca = PCA(n_components=10, random_state=42)
pca_transformer = Pipeline([
    ("features", featureTransformer),
    ("pca", pca) 
])

def getPCA(df):
    transformed_PCA_data = pca.fit_transform(df)
    return transformed_PCA_data

# KNeighbours Classifier

In [63]:
kn_model = KNeighborsClassifier()
kn_model.fit(X_train_encoded, y_train)
kn_result = kn_model.predict(X_test_encoded)
kn_accuracy = accuracy_score(y_test, kn_result)
print(kn_accuracy)

0.3142394345457223


In [64]:
kn_model = KNeighborsClassifier()
kn_model.fit(getPCA(X_train_encoded), y_train)
kn_result = kn_model.predict(getPCA(X_test_encoded))
kn_accuracy = accuracy_score(y_test, kn_result)
print("After PCA: ", kn_accuracy)

After PCA:  0.3068767486379031


# Random Forest Classifier

In [65]:
rf_classifier_model = RandomForestClassifier(random_state=42)
rf_classifier_model.fit(X_train_encoded, y_train)
rf_result = rf_classifier_model.predict(X_test_encoded)
rf_accuracy = accuracy_score(y_test, rf_result)
print(rf_accuracy)
print(type(rf_classifier_model))

0.40156088941245766
<class 'sklearn.ensemble._forest.RandomForestClassifier'>


In [66]:
rf_classifier_model_pca = RandomForestClassifier(random_state=42)
rf_classifier_model_pca.fit(getPCA(X_train_encoded), y_train)
rf_result_pca = rf_classifier_model_pca.predict(getPCA(X_test_encoded))
rf_accuracy_pca = accuracy_score(y_test, rf_result_pca)
print("After PCA: ",rf_accuracy_pca)

After PCA:  0.3626859078191724


In [67]:
param_grid = {
    'n_estimators': [100,200,350],  # Number of trees in the forest
    'max_depth': [20, 23, 25],  # Maximum depth of the tree
    'min_samples_split': [10,15, 20],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 3, 5]     # Minimum number of samples required to be at a leaf node
}

# Set up GridSearchCV
grid_search_rf = GridSearchCV(estimator=rf_classifier_model, param_grid=param_grid, 
                           scoring='accuracy', cv=5, verbose=2, n_jobs=-1)

print(X_train_encoded.shape)
print(X_test_encoded.shape)
# Fit GridSearchCV to the data
grid_search_rf.fit(X_train_encoded, y_train)

# Get the best parameters and the corresponding accuracy
best_params = grid_search_rf.best_params_
best_accuracy = grid_search_rf.best_score_

print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy}")

(27160, 91)
(6791, 91)
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters: {'max_depth': 25, 'min_samples_leaf': 1, 'min_samples_split': 15, 'n_estimators': 350}
Best accuracy: 0.4171575846833579


In [68]:
improved_rf_classifier_model = grid_search_rf.best_estimator_
improved_rf_result = improved_rf_classifier_model.predict(X_test_encoded)
improved_rf_accuracy = accuracy_score(y_test, improved_rf_result)
print(improved_rf_accuracy)

0.4146664703283758


# SVM

In [69]:
svm_model = SVC(random_state=42)
svm_model.fit(X_train_encoded, y_train)
svm_result = svm_model.predict(X_test_encoded)
svm_accuracy = accuracy_score(y_test, svm_result)
print(svm_accuracy)

0.3922839051686055


In [70]:
svm_model = SVC(random_state=42)
svm_model.fit(getPCA(X_train_encoded), y_train)
svm_result = svm_model.predict(getPCA(X_test_encoded))
svm_accuracy = accuracy_score(y_test, svm_result)
print("After PCA", svm_accuracy)

After PCA 0.38403769695184803


# Gradient Boosting Classifier

In [71]:
gradient_boost_model = GradientBoostingClassifier(random_state=42)
gradient_boost_model.fit(X_train_encoded, y_train)
gradient_boost_result = gradient_boost_model.predict(X_test_encoded)
gradient_boost_accuracy = accuracy_score(y_test, gradient_boost_result)
print(gradient_boost_accuracy)

0.42070387277278753


In [72]:
gradient_boost_model_pca = GradientBoostingClassifier(random_state=42)
gradient_boost_model_pca.fit(getPCA(X_train_encoded), y_train)
gradient_boost_result_pca = gradient_boost_model_pca.predict(getPCA(X_test_encoded))
gradient_boost_accuracy_pca = accuracy_score(y_test, gradient_boost_result)
print("After PCA: ",gradient_boost_accuracy_pca)

After PCA:  0.42070387277278753


In [73]:
param_grid = {
    'n_estimators': [100,200],   # Number of boosting stages to be run
    'learning_rate': [0.1, 0.3, 0.5], # Learning rate shrinks the contribution of each tree
    'max_depth': [5, 10, 20],            # Maximum depth of the individual regression estimators
    'min_samples_split': [2],    # Minimum number of samples required to split an internal node
    'min_samples_leaf': [4, 6]      # Minimum number of samples required to be at a leaf node
}

grid_search_gb = GridSearchCV(estimator=gradient_boost_model, param_grid=param_grid, 
                              cv=2, n_jobs=-1, verbose=2)

# Fit GridSearchCV on the data
grid_search_gb.fit(X_train_encoded, y_train)

# The best parameters found by GridSearchCV
print("Best Parameters:", grid_search_gb.best_params_)
print(f"Best accuracy: {best_accuracy}")

Fitting 2 folds for each of 36 candidates, totalling 72 fits
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   4.2s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=350; total time=   7.4s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=15, n_estimators=350; total time=   6.7s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=20, n_estimators=350; total time=   6.6s
[CV] END max_depth=20, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   3.8s
[CV] END max_depth=20, min_samples_leaf=3, min_samples_split=15, n_estimators=100; total time=   1.7s
[CV] END max_depth=20, min_samples_leaf=3, min_samples_split=15, n_estimators=100; total time=   1.7s
[CV] END max_depth=20, min_samples_leaf=3, min_samples_split=15, n_estimators=200; total time=   3.4s
[CV] END max_depth=20, min_samples_leaf=3, min_samples_split=20, n_estimators=100; total time=   1.7s
[CV] END max_depth=20

Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 6, 'min_samples_split': 2, 'n_estimators': 100}
Best accuracy: 0.4171575846833579


In [74]:
improved_gradient_boost_model = grid_search_gb.best_estimator_
improved_gradient_boost_result = improved_gradient_boost_model.predict(X_test_encoded)
improved_gradient_boost_accuracy = accuracy_score(y_test, improved_gradient_boost_result)
print(improved_gradient_boost_accuracy)

0.4139302017375939


# Adaboost

In [75]:
adaboost_model = AdaBoostClassifier(random_state=42)
adaboost_model.fit(X_train_encoded, y_train)
adaboost_result = adaboost_model.predict(X_test_encoded)
adaboost_model_accuracy = accuracy_score(y_test, adaboost_result)
print(adaboost_model_accuracy)

0.3893388308054778


In [76]:
adaboost_model_pca = AdaBoostClassifier(random_state=42)
adaboost_model_pca.fit(getPCA(X_train_encoded), y_train)
adaboost_model_pca = adaboost_model_pca.predict(getPCA(X_test_encoded))
adaboost_model_accuracy_pca = accuracy_score(y_test, gradient_boost_result)
print("After PCA: ",adaboost_model_accuracy_pca)

After PCA:  0.42070387277278753
